In [ ]:
%cd ..
#import site
#site.addsitedir('C:/Users/Admin/Documents/GitHub/blokus/')  # Replace with the actual path to your project folder
from src import bot
import src.objects as o
from src.objects import Player, Piece
from pygame import display, Surface, font, image, surfarray, Rect
import pygame
import src.events as e
import src.views as v
from os.path import join
import src.controllers as c
import numpy as np

In [ ]:
board = o.LinkedGrid(20,20, 20)
players = []
colors = ["r", "g", "b", "y"]
for p in range(4):
    players.append(Player(colors[p]))
player = players[0]

In [ ]:
player.all_playable_moves[(0,0)][100][2]
for i, piece in enumerate(player.pieces[5]):
    if piece.m.tolist() == player.all_playable_moves[(0,0)][100][2]:
        print(i)

In [ ]:
playable_board, playable_pos = bot.get_playable_conditions(bot.convert_matrix_to_nparray(board.matrix),bot.convert_color_to_number(player.c))


In [ ]:
bot.check_absolute_coordinates(playable_board, [(0,0),(0,1)])

In [ ]:
playable_pos = [(i,j) for i in range(20) for j in range(20)]

In [ ]:
all_pos_play = {}
for pkey in player.pieces.keys():
    for pindex in range(len(player.pieces[pkey])):
        piece = player.pieces[pkey][pindex].m
        piece_permutations = [piece, np.rot90(piece,1), np.rot90(piece,2), np.rot90(piece,3), np.fliplr(piece), np.rot90(np.fliplr(piece),1), np.rot90(np.fliplr(piece),2), np.rot90(np.fliplr(piece),3)]  
        for p in piece_permutations:
            for position in playable_pos:
                for i in range(len(p)):
                    for j in range(len(p[0])):
                        relative_coords = bot.get_piece_relative_coordinates(p, i,j)
                        absolute_coords = bot.get_absolute_coordinates(position, relative_coords)
                        if position in absolute_coords:
                            if position not in all_pos_play.keys():
                                 all_pos_play[position] = []
                                 
                            if bot.check_absolute_coordinates(playable_board, absolute_coords):
                                if [(pkey,pindex),absolute_coords] not in all_pos_play[position]:
                                    all_pos_play[position].append([(pkey,pindex),absolute_coords,piece.tolist()])

In [ ]:
all_pos_play[(0,0)]

In [ ]:
import json

"""
# Specify the file path where you want to save the JSON file
file_path = 'data/all_playable_moves.json'
string_keys_dict = {str(key): value for key, value in all_pos_play.items()}
# Open the file in write mode and use json.dump() to save the list as a JSON file
with open(file_path, 'w') as json_file:
    json.dump(string_keys_dict, json_file)"""

In [ ]:
file_path = 'data/all_playable_moves.json'

# Open the file in read mode and use json.load() to load the dictionary from the JSON file
with open(file_path, 'r') as json_file:
    string_keys_dict = json.load(json_file)

# Convert string keys back to tuples
loaded_dict = {tuple(eval(key)): value for key, value in string_keys_dict.items()}

print('The dictionary with tuple keys has been loaded:')
print(loaded_dict)

In [ ]:
loaded_dict[(0,0)]

In [ ]:
a = [[0, 0], [0, 1], None, None, [1, 1], [1, 2], None, [2, 1], None]
print(a[2])

In [ ]:
[None if lst is None else tuple(lst) for lst in [[0, 0], [0, 1], None, None, [1, 1], [1, 2], None, [2, 1], None]]

In [ ]:
bot.check_absolute_coordinates(playable_board, loaded_dict[(0,0)][50][1])


In [ ]:
# Sample list of sublists
list_of_sublists = [(("a", 1), [1, 2, 3]), (("b", 2), [4, 5, 6]), (("a", 1), [7, 8, 9]), (("c", 3), [10, 11, 12])]

# Tuple to remove
tuple_to_remove = ("a", 1)

# Use list comprehension to filter out sublists where the first element is the given tuple
filtered_list = [sublist for sublist in list_of_sublists if sublist[0] != tuple_to_remove]

print(filtered_list)


In [ ]:
a=1
b=a
a=2
print(a,b)

In [ ]:
a = player.pieces[5][5].m

In [ ]:
for i, piece in enumerate(player.pieces[5]):
    if piece.m is a:
        index = i
        break

In [ ]:
index

In [ ]:
np.array_equal(player.all_pieces[5][2].m,player.pieces[5][2].m)

In [ ]:
player.all_pieces[5][2].m

In [ ]:
player.pieces[5][2].m

In [ ]:
len(player.all_playable_moves[(1,0)])

In [ ]:
player.updateAllMoves((5,9))

In [ ]:
player.all_playable_moves[(1,0)][0][1]

In [1]:
%cd ..

import src.objects as o
from src.objects import Player, Piece
from pygame import display, Surface, font, image, surfarray, Rect
import pygame
import src.events as e
import src.views as v
from os.path import join
import src.controllers as c
import sys



c:\Users\Admin\Documents\GitHub\blokus
pygame 2.5.2 (SDL 2.28.3, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [93]:
import numpy as np

In [70]:
from src import bot
board = o.LinkedGrid(20,20, 20)
players = []
colors = ["r", "g", "b", "y"]
for p in range(4):
    players.append(Player(colors[p]))

In [71]:
p_i=0

In [76]:
run = True
weights = [1,1,10]
turn = 0

print('now plays ',p_i )
player = players[p_i]


if False not in [player.isDone for player in players]: # when all players are done, game stops.
    run = False
    

if not player.isDone:
    matrix = bot.convert_matrix_to_nparray(board.matrix)
    possible_places, possible_plays_indices, possible_pieces = bot.get_available_actions(matrix, player)
    print(p_i,player.c, possible_plays_indices)
    possible_places = [None if lst is None else tuple(lst) for lst in possible_places]
    if len(possible_plays_indices) == 0:
        player.isDone = True
    else:
        # Random play right now.
        best = 0
        choice = 0
        best_reward = None

        for i in range(len(possible_places)):
            reward = bot.estimate_rewards(matrix, possible_places[i], bot.convert_color_to_number(player.c))
            current_sum = sum([x * y for x, y in zip(weights, reward)])
            if current_sum > best:
                choice = i
                best = current_sum
                best_reward = reward

        # rand_index = bot.random_index(possible_plays_indices)
        pkey, pindex = possible_plays_indices[choice]
        absolute_coords = possible_places[choice]
        piece_played = possible_pieces[choice]
        #print(choice)
        bot.play_coordinates(pkey, pindex, absolute_coords, piece_played, player, board)
        if len(player.pieces) == 0:
            player.isDone = True
        print(bot.convert_matrix_to_nparray(board.matrix))

print('now changing player')
p_i+=1
if p_i>3: 
    p_i=0

now plays  0
0 r []
now changing player


In [77]:
print(bot.convert_matrix_to_nparray(board.matrix))

[[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2]
 [0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0]
 [3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 4 0]
 [3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 4]]


In [119]:
player = players[0]
get_available_actions(matrix, player)

[(0, 3), (2, 3), (3, 0), (3, 2)]
[[0, 3]]
[[0, 3]]
[[0, 3]]
[[0, 3]]
[[0, 3]]
[[0, 3]]
[[0, 3]]
[[0, 3]]
[[0, 3], [1, 3]]
[[0, 3], [0, 4]]
[[0, 2], [0, 3]]
[[0, 3], [1, 3]]
[[0, 3], [0, 4]]
[[0, 2], [0, 3]]
[[0, 3], [1, 3]]
[[0, 3], [0, 4]]
[[0, 2], [0, 3]]
[[0, 3], [1, 3]]
[[0, 3], [0, 4]]
[[0, 2], [0, 3]]
[[0, 3], [1, 3], [2, 3]]
[[0, 3], [0, 4], [0, 5]]
[[0, 2], [0, 3], [0, 4]]
[[0, 1], [0, 2], [0, 3]]
[[0, 3], [1, 3], [2, 3]]
[[0, 3], [0, 4], [0, 5]]
[[0, 2], [0, 3], [0, 4]]
[[0, 1], [0, 2], [0, 3]]
[[0, 3], [1, 3], [2, 3]]
[[0, 3], [0, 4], [0, 5]]
[[0, 2], [0, 3], [0, 4]]
[[0, 1], [0, 2], [0, 3]]
[[0, 3], [1, 3], [2, 3]]
[[0, 3], [0, 4], [0, 5]]
[[0, 2], [0, 3], [0, 4]]
[[0, 1], [0, 2], [0, 3]]
[[0, 3], None, [1, 3], [1, 4]]
[None, [0, 3], [1, 2], [1, 3]]
[[0, 3], [0, 4], None, [1, 4]]
[[0, 2], [0, 3], None, [1, 3]]
[[0, 3], [0, 4], [1, 3], None]
[[0, 2], [0, 3], [1, 2], None]
[None, [0, 3], [1, 2], [1, 3]]
[[0, 3], [0, 4], None, [1, 4]]
[[0, 2], [0, 3], None, [1, 3]]
[[0, 3], [0,

([], [], [])

In [118]:
def get_available_actions(matrix, player):
    # Return a list of valid moves as a form of coordinates to play and piece indices
    possible_plays_indices = []
    possible_places = []
    possible_pieces = []
    playable_board, playable_pos = bot.get_playable_conditions(matrix,bot.convert_color_to_number(player.c))
    print(playable_pos)
    for position in playable_pos:
        #print(player.c, position, len(player.all_playable_moves[position]))
        for move in player.all_playable_moves[position]:
            #print(move[1])
            absolute_coords = move[1]
            #print(absolute_coords)
            if check_absolute_coordinates(playable_board, absolute_coords):
                possible_places.append(absolute_coords)
                possible_plays_indices.append(move[0])
                possible_pieces.append(move[2])
                                    
    return possible_places, possible_plays_indices, possible_pieces

In [117]:
def check_absolute_coordinates(matrix, absolute_coords):
    rows, cols = np.shape(matrix)
    placeable = True
    for abs_coord in absolute_coords:
            print(absolute_coords,abs_coord)
            if abs_coord is not None:
                print('here')
                if 0 <= abs_coord[0] < rows and 0 <= abs_coord[1] < cols:
                    print('double_here')
                    if matrix[abs_coord[0]][abs_coord[1]] > 0:
                        print('triple here')
                        placeable = False
                        break
                         # Found a non-empty cell at the specified position
                else:# Coordinates are out of the board range
                    placeable = False
                    break  
    return placeable  # All specified positions are in range and are empty

In [ ]:
a,b,c = get_available_actions(matrix, player)

In [ ]:
c